In [1]:
import numpy as np
import matplotlib.pyplot as plt
import anacal
from deep_anacal import simulate, utils
%load_ext autoreload
%autoreload 2

In [2]:
seed = 2
ngrid = 64
scale = 0.2
indx = [32]
indy = [32]
ns = len(indx) * len(indy)
inds = np.meshgrid(indy, indx, indexing="ij")
yx = np.vstack([np.ravel(_) for _ in inds])
dtype = np.dtype(
    [
        ("y", np.int32),
        ("x", np.int32),
        ("is_peak", np.int32),
        ("mask_value", np.int32),
    ]
)
detection = np.empty(ns, dtype=dtype)
detection["y"] = yx[0]
detection["x"] = yx[1]
detection["is_peak"] = np.ones(ns)
detection["mask_value"] = np.zeros(ns)

In [3]:
acal_res = []
fpfs_config = anacal.fpfs.FpfsConfig(
    sigma_arcsec=0.52
)
noise_std = 1e-3
for shear in [0.02, -0.02]:
    gal_array, psf_array = simulate.simulate_exponential(
        seed=seed,
        ngrid=ngrid,
        scale=scale,
        g1=shear,
        g2=0.0,
    )
    acal_res.append(
        anacal.fpfs.process_image(
            fpfs_config=fpfs_config,
            mag_zero=30.0,
            gal_array=gal_array,
            psf_array=psf_array,
            pixel_scale=scale,
            noise_variance=noise_std**2,
            noise_array=None,
            detection=detection,
        )
    )

In [4]:
ep, Rp, em, Rm = utils.get_e_R(wacal_res=acal_res, dacal_res=acal_res, force_detection=True)
m, merr, c, cerr = utils.compute_m_and_c(
    e_plus=np.sum(ep),
    R_plus=np.sum(Rp),
    e_minus=np.sum(em),
    R_minus=np.sum(Rm),
)
print("m: %f [1e-3]" % (m/1e-3), flush=True)
print("c: %f [1e-4]" % (c/1e-4), flush=True)

m: 0.621624 [1e-3]
c: 0.000224 [1e-4]
